In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# Load data from the single CSV file
data1 = pd.read_csv('data/malicious_phish.csv')
data1.head(10)


,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement
5,http://buzzfil.net/m/show-art/ils-etaient-loin...,benign
6,espn.go.com/nba/player/_/id/3457/brandon-rush,benign
7,yourbittorrent.com/?q=anthony-hamilton-soulife,benign
8,http://www.pashminaonline.com/pure-pashminas,defacement
9,allmusic.com/album/crazy-from-the-heat-r16990,benign


In [2]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651191 entries, 0 to 651190
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     651191 non-null  object
 1   type    651191 non-null  object
dtypes: object(2)
memory usage: 9.9+ MB


In [3]:
data1.type.unique()

array(['phishing', 'benign', 'defacement', 'malware'], dtype=object)

In [4]:
data = data1.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651191 entries, 0 to 651190
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     651191 non-null  object
 1   type    651191 non-null  object
dtypes: object(2)
memory usage: 9.9+ MB


In [5]:
# Ensure that the 'text' column contains only strings (not floats)
data = data.dropna(subset=['url'])
data['url'] = data['url'].astype(str)

# Text preprocessing function for fake news articles
def preprocess_news(text):
    # Customize text preprocessing here
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation and numbers
    text = ''.join([char for char in text if char not in string.punctuation and not char.isdigit()])
    
    # Tokenization (split text into words)
    tokens = word_tokenize(text)
    
    # Remove stopwords (common words like "the", "and", "is")
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Rejoin tokens into a single string
    text = ' '.join(tokens)
    
    return text

# Apply text preprocessing to the 'text' column
data['url'] = data['url'].apply(preprocess_news)

# Create a TF-IDF vectorizer to convert text data into feature vectors
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(data['url'].values)
y = data['url'].values
joblib.dump(vectorizer, 'linksTfidfVectorizer.sav')

# Split the dataset into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

# Define the batch size
batch_size = 1000  # You can adjust this based on your available memory and processing power

# Calculate the number of batches
num_batches = len(X_train) // batch_size + 1

# Initialize your SVM classifier
svm_classifier = SVC(probability=True)

# Initialize variables to store evaluation metrics
total_predictions = []
total_true_labels = []

# Train the model in batches
for batch in range(num_batches):
    start_idx = batch * batch_size
    end_idx = (batch + 1) * batch_size
    X_batch = X_train[start_idx:end_idx]
    y_batch = y_train[start_idx:end_idx]

    # Fit the SVM classifier on the current batch
    svm_classifier.fit(X_batch, y_batch)

    # Make predictions on the test set (or a validation set if you prefer)
    test_predictions = svm_classifier.predict(X_test)

    # Append batch predictions and true labels to the total lists
    total_predictions.extend(test_predictions)
    total_true_labels.extend(y_test)

# Calculate evaluation metrics for the entire dataset
svm_accuracy = accuracy_score(total_true_labels, total_predictions)
svm_precision = precision_score(total_true_labels, total_predictions, average='macro', labels=data['label'].unique())
svm_recall = recall_score(total_true_labels, total_predictions, average='macro', labels=data['label'].unique())
svm_f1 = f1_score(total_true_labels, total_predictions, average='macro', labels=data['label'].unique())

# Print results for the entire dataset
print("\nSVM Classifier:")
print(f"Accuracy: {svm_accuracy}")
print(f"Precision: {svm_precision}")
print(f"Recall: {svm_recall}")
print(f"F1 Score: {svm_f1}")

# Plot confusion matrix for the entire dataset
plt.figure(figsize=(12, 6))
svm_cm = confusion_matrix(total_true_labels, total_predictions, labels=data['label'].unique())
sns.heatmap(svm_cm, annot=True, fmt='d', cmap='Blues', xticklabels=data['label'].unique(), yticklabels=data['label'].unique())
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix for SVM (Entire Dataset)')
plt.tight_layout()
plt.show()

# Save the trained SVM classifier
joblib.dump(svm_classifier, 'linksmodel.sav')